## Collect, annotate, and evaluate person detection using YOLO.

##### Install & import libraries

In [12]:
#!pip install ultralytics
#!pip3 install labelImg
#!pip install torchmetrics
#!pip install lxml


In [16]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

In [2]:
import os
import subprocess
import requests
import random
import cv2
import numpy as np
from ultralytics import YOLO 
import torch
from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [3]:
try:
    import labelImg
except ImportError:
    print("Please install LabelImg from https://github.com/tzutalin/labelImg")

### 1. Collected data using API

In [4]:
def collect_images(query="people", output_dir="dataset/images", num_images=100):
    os.makedirs(output_dir, exist_ok=True)
    api_key = "unwSPFuEVRy9xS7Rd46VoExpSeYLwUX0macOV5xOEwoH17Kw0sNyIGax"  
    url = "https://api.pexels.com/v1/search"
    headers = {"Authorization": api_key}

    per_page = 15
    pages = (num_images // per_page) + 1

    count = 0
    for page in range(1, pages + 1):
        params = {"query": query, "per_page": per_page, "page": page}
        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        for photo in data["photos"]:
            if count >= num_images:
                break
            img_url = photo["src"]["original"]
            img_data = requests.get(img_url).content
            with open(os.path.join(output_dir, f"image_{count + 1}.jpg"), "wb") as f:
                f.write(img_data)
            count += 1
            print(f"Downloaded image {count}")

    print(f"Downloaded {count} images to {output_dir}")

collect_images(query="people in diverse conditions", output_dir="dataset/images", num_images=100)


Downloaded image 1
Downloaded image 2
Downloaded image 3
Downloaded image 4
Downloaded image 5
Downloaded image 6
Downloaded image 7
Downloaded image 8
Downloaded image 9
Downloaded image 10
Downloaded image 11
Downloaded image 12
Downloaded image 13
Downloaded image 14
Downloaded image 15
Downloaded image 16
Downloaded image 17
Downloaded image 18
Downloaded image 19
Downloaded image 20
Downloaded image 21
Downloaded image 22
Downloaded image 23
Downloaded image 24
Downloaded image 25
Downloaded image 26
Downloaded image 27
Downloaded image 28
Downloaded image 29
Downloaded image 30
Downloaded image 31
Downloaded image 32
Downloaded image 33
Downloaded image 34
Downloaded image 35
Downloaded image 36
Downloaded image 37
Downloaded image 38
Downloaded image 39
Downloaded image 40
Downloaded image 41
Downloaded image 42
Downloaded image 43
Downloaded image 44
Downloaded image 45
Downloaded image 46
Downloaded image 47
Downloaded image 48
Downloaded image 49
Downloaded image 50
Downloade

### 2. Annotate the data

In [28]:
annotations_dir = "dataset/labels"
os.makedirs(annotations_dir, exist_ok=True)

print("Annotate the images using LabelImg and save annotations in YOLO format.")
print(f"Save your annotations in: {annotations_dir}")


Annotate the images using LabelImg and save annotations in YOLO format.
Save your annotations in: dataset/labels


###3.Converted .xml file into .txt file

In [6]:
import xml.etree.ElementTree as ET

def convert_voc_to_yolo(xml_dir, output_dir, labels):
    """
    Converts PASCAL VOC annotations (.xml) to YOLO format (.txt).

    Args:
        xml_dir (str): Path to the directory containing .xml files.
        output_dir (str): Path to save YOLO .txt files.
        labels (list): List of class names (in order).
    """
    os.makedirs(output_dir, exist_ok=True)

    for xml_file in os.listdir(xml_dir):
        if not xml_file.endswith('.xml'):
            continue

        xml_path = os.path.join(xml_dir, xml_file)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        size = root.find('size')
        width = int(size.find('width').text)
        height = int(size.find('height').text)

        yolo_annotations = []
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            if class_name not in labels:
                continue
            class_id = labels.index(class_name)

            bbox = obj.find('bndbox')
            xmin = float(bbox.find('xmin').text)
            ymin = float(bbox.find('ymin').text)
            xmax = float(bbox.find('xmax').text)
            ymax = float(bbox.find('ymax').text)

            # Convert to YOLO format
            x_center = ((xmin + xmax) / 2) / width
            y_center = ((ymin + ymax) / 2) / height
            box_width = (xmax - xmin) / width
            box_height = (ymax - ymin) / height

            yolo_annotations.append(f"{class_id} {x_center} {y_center} {box_width} {box_height}")

        txt_file = os.path.join(output_dir, os.path.splitext(xml_file)[0] + '.txt')
        with open(txt_file, 'w') as f:
            f.write('\n'.join(yolo_annotations))

xml_directory = "dataset/labels_xml"
yolo_output_directory = "dataset/labels"
class_labels = ["person"]  

convert_voc_to_yolo(xml_dir= r"C:\Users\sandh.SHRIHARI\OneDrive\Desktop\Data_science\YOLO\dataset\labels", output_dir=yolo_output_directory, labels=class_labels)
print("Conversion complete!")


Conversion complete!


### 4. Run YOLO model

In [15]:
runs_path = r"C:\Users\sandh.SHRIHARI\OneDrive\Desktop\Data_science\YOLO\runs"

os.makedirs(runs_path, exist_ok=True)
print(f"Directory created: {runs_path}")

Directory created: C:\Users\sandh.SHRIHARI\OneDrive\Desktop\Data_science\YOLO\runs


In [17]:
def run_yolo_model(images_dir="dataset/images", model_path="yolov8n.pt", output_dir="runs/detect"):
    """
    Run a pre-trained YOLO model on the annotated images and handle empty predictions.
    Saves results in the specified output directory.
    """

    os.makedirs(output_dir, exist_ok=True)

    model = YOLO(model_path)

    image_files = [os.path.join(images_dir, f) for f in os.listdir(images_dir) if f.endswith('.jpg')]

    if not image_files:
        raise ValueError("No image files found in the specified directory.")

    try:
        results = model.predict(
            source=images_dir,  
            save=True,         
            save_txt=True,      
            project=output_dir, 
            name="detect"       )
        print(f"Results saved in: {os.path.join(output_dir, 'detect')}")
    except Exception as e:
        print(f"Error during model prediction: {e}")
        raise

    return results


### 5. Evaluate metrics using Torchmetrics

In [29]:
def evaluate_metrics(annotations_dir="dataset/labels", predictions_dir="runs/detect/detect"):
    """
    Compute Precision, Recall, F1 Score, and mAP@50 using torchmetrics.
    """
    ground_truths = []
    predictions = []
    for label_file in os.listdir(annotations_dir):
        label_path = os.path.join(annotations_dir, label_file)
        with open(label_path, 'r') as f:
            bboxes = []
            labels = []
            for line in f.readlines():
                parts = list(map(float, line.strip().split()))
                labels.append(int(parts[0]))  # Class ID
                bbox = [
                    parts[1] - parts[3] / 2,  # x_min
                    parts[2] - parts[4] / 2,  # y_min
                    parts[1] + parts[3] / 2,  # x_max
                    parts[2] + parts[4] / 2,  # y_max
                ]
                bboxes.append(bbox)
            ground_truths.append({"boxes": torch.tensor(bboxes), "labels": torch.tensor(labels)})

    for prediction_file in os.listdir(predictions_dir):
        prediction_path = os.path.join(predictions_dir, prediction_file)
        with open(prediction_path, 'r') as f:
            bboxes = []
            scores = []
            labels = []
            for line in f.readlines():
                parts = list(map(float, line.strip().split()))
                labels.append(int(parts[0]))  
                scores.append(parts[5])       
                bbox = [
                    parts[1] - parts[3] / 2,  # x_min
                    parts[2] - parts[4] / 2,  # y_min
                    parts[1] + parts[3] / 2,  # x_max
                    parts[2] + parts[4] / 2,  # y_max
                ]
                bboxes.append(bbox)
            predictions.append({"boxes": torch.tensor(bboxes), "scores": torch.tensor(scores), "labels": torch.tensor(labels)})

    while len(predictions) < len(ground_truths):
        predictions.append({"boxes": torch.empty((0, 4)), "scores": torch.empty((0,)), "labels": torch.empty((0,), dtype=torch.long)})

    if len(predictions) != len(ground_truths):
        raise ValueError("Mismatch in number of ground truths and predictions!")

    metric = MeanAveragePrecision()
    metric.update(predictions, ground_truths)

    result = metric.compute()
    print("Evaluation Results:")
    print(f"Precision: {result['map']:.4f}")
    print(f"Recall: {result['map50']:.4f}")
    print(f"F1 Score: Calculated separately")
    print(f"mAP@50: {result['map50']:.4f}")

    return result


### 6. Main script to define full pipeline 

In [20]:
# Define paths
images_dir = "dataset/images"
annotations_dir = "dataset/labels"
model_path = "yolov8n.pt"


In [21]:
try:
    collect_images(output_dir=images_dir, num_images=100)
except Exception as e:
    print(f"Error in collecting images: {e}")


Downloaded image 1
Downloaded image 2
Downloaded image 3
Downloaded image 4
Downloaded image 5
Downloaded image 6
Downloaded image 7
Downloaded image 8
Downloaded image 9
Downloaded image 10
Downloaded image 11
Downloaded image 12
Downloaded image 13
Downloaded image 14
Downloaded image 15
Downloaded image 16
Downloaded image 17
Downloaded image 18
Downloaded image 19
Downloaded image 20
Downloaded image 21
Downloaded image 22
Downloaded image 23
Downloaded image 24
Downloaded image 25
Downloaded image 26
Downloaded image 27
Downloaded image 28
Downloaded image 29
Downloaded image 30
Downloaded image 31
Downloaded image 32
Downloaded image 33
Downloaded image 34
Downloaded image 35
Downloaded image 36
Downloaded image 37
Downloaded image 38
Downloaded image 39
Downloaded image 40
Downloaded image 41
Downloaded image 42
Downloaded image 43
Downloaded image 44
Downloaded image 45
Downloaded image 46
Downloaded image 47
Downloaded image 48
Downloaded image 49
Downloaded image 50
Downloade

In [22]:
import shutil
def batch_process_images(input_dir, batch_size, process_function, *args, **kwargs):
    """
    Process images in smaller batches to avoid memory overload.
    
    Args:
        input_dir (str): Path to the directory containing images.
        batch_size (int): Number of images to process in each batch.
        process_function (callable): Function to process the batch of images.
        *args: Additional positional arguments for the process function.
        **kwargs: Additional keyword arguments for the process function.
    """
    image_files = [f for f in os.listdir(input_dir) if f.endswith((".jpg", ".png"))]
    total_images = len(image_files)
    
    print(f"Total images: {total_images}. Processing in batches of {batch_size}.")
    
    for i in range(0, total_images, batch_size):
        batch = image_files[i:i + batch_size]
        print(f"Processing batch {i // batch_size + 1}: {len(batch)} images")
  
        temp_batch_dir = os.path.join(input_dir, "temp_batch")
        os.makedirs(temp_batch_dir, exist_ok=True)
 
        for img_file in batch:
            shutil.copy(os.path.join(input_dir, img_file), os.path.join(temp_batch_dir, img_file))

        try:
            process_function(temp_batch_dir, *args, **kwargs)
        except Exception as e:
            print(f"Error processing batch {i // batch_size + 1}: {e}")

        shutil.rmtree(temp_batch_dir)
        print(f"Completed batch {i // batch_size + 1}/{(total_images // batch_size) + 1}")
    
    print("Batch processing completed.")

def dummy_process_function(temp_dir):
    """
    Dummy function to simulate processing of images in a batch.
    Replace this with your actual image processing function, like YOLO detection.
    """
    print(f"Processing images in directory: {temp_dir}")
    for img_file in os.listdir(temp_dir):
        print(f"Processed {img_file}")

batch_process_images("dataset/images", batch_size=10, process_function=dummy_process_function)


Total images: 100. Processing in batches of 10.
Processing batch 1: 10 images
Processing images in directory: dataset/images\temp_batch
Processed image_1.jpg
Processed image_10.jpg
Processed image_100.jpg
Processed image_11.jpg
Processed image_12.jpg
Processed image_13.jpg
Processed image_14.jpg
Processed image_15.jpg
Processed image_16.jpg
Processed image_17.jpg
Completed batch 1/11
Processing batch 2: 10 images
Processing images in directory: dataset/images\temp_batch
Processed image_18.jpg
Processed image_19.jpg
Processed image_2.jpg
Processed image_20.jpg
Processed image_21.jpg
Processed image_22.jpg
Processed image_23.jpg
Processed image_24.jpg
Processed image_25.jpg
Processed image_26.jpg
Completed batch 2/11
Processing batch 3: 10 images
Processing images in directory: dataset/images\temp_batch
Processed image_27.jpg
Processed image_28.jpg
Processed image_29.jpg
Processed image_3.jpg
Processed image_30.jpg
Processed image_31.jpg
Processed image_32.jpg
Processed image_33.jpg
Proc

### 7. Used batch process because of overloading

In [23]:
def yolo_process_batch(batch_dir, model_path):
    """
    Process a batch of images using YOLO.
    
    Args:
        batch_dir (str): Path to the batch directory containing images.
        model_path (str): Path to the YOLO model file.
    """
    run_yolo_model(images_dir=batch_dir, model_path=model_path)

batch_process_images("dataset/images", batch_size=10, process_function=yolo_process_batch, model_path="yolov8n.pt")


Total images: 100. Processing in batches of 10.
Processing batch 1: 10 images

image 1/10 C:\Users\sandh.SHRIHARI\OneDrive\Desktop\Data_science\YOLO\dataset\images\temp_batch\image_1.jpg: 448x640 1 person, 451.5ms
image 2/10 C:\Users\sandh.SHRIHARI\OneDrive\Desktop\Data_science\YOLO\dataset\images\temp_batch\image_10.jpg: 384x640 9 persons, 560.4ms
image 3/10 C:\Users\sandh.SHRIHARI\OneDrive\Desktop\Data_science\YOLO\dataset\images\temp_batch\image_100.jpg: 416x640 4 persons, 1 sports ball, 1 chair, 1 laptop, 357.2ms
image 4/10 C:\Users\sandh.SHRIHARI\OneDrive\Desktop\Data_science\YOLO\dataset\images\temp_batch\image_11.jpg: 448x640 1 apple, 2 oranges, 1 vase, 723.0ms
image 5/10 C:\Users\sandh.SHRIHARI\OneDrive\Desktop\Data_science\YOLO\dataset\images\temp_batch\image_12.jpg: 448x640 11 persons, 1 backpack, 1 umbrella, 478.1ms
image 6/10 C:\Users\sandh.SHRIHARI\OneDrive\Desktop\Data_science\YOLO\dataset\images\temp_batch\image_13.jpg: 448x640 1 person, 274.7ms
image 7/10 C:\Users\sandh

In [25]:
annotations_dir = r"C:\Users\sandh.SHRIHARI\OneDrive\Desktop\Data_science\YOLO\dataset\labels"
predictions_dir = r"C:\Users\sandh.SHRIHARI\OneDrive\Desktop\Data_science\YOLO\runs\detect\detect"

# Ensure directories exist
if not os.path.exists(annotations_dir):
    raise FileNotFoundError(f"Annotations directory not found: {annotations_dir}")
if not os.path.exists(predictions_dir):
    raise FileNotFoundError(f"Predictions directory not found: {predictions_dir}")

print("Directories verified. Proceeding with evaluation.")

Directories verified. Proceeding with evaluation.


In [30]:
try:
    evaluation_results = evaluate_metrics(annotations_dir=annotations_dir, predictions_dir=r"C:\Users\sandh.SHRIHARI\OneDrive\Desktop\Data_science\YOLO\runs\detect")
    print(evaluation_results)
except Exception as e:
    print(f"Error in evaluating results: {e}")


Error in evaluating results: [Errno 13] Permission denied: 'C:\\Users\\sandh.SHRIHARI\\OneDrive\\Desktop\\Data_science\\YOLO\\runs\\detect\\detect'


##### Tried to grant permission ...but it either completely shutting down the windows or if i choose to move this file then also getting permission denied error , i've tried to recreate the folder from scratch but coz of the yolo model windows not working properly ...It's totally getting hang